### Praktikum 3

In [ ]:
!pip install hnswlib -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import hnswlib
import numpy as np
import time
from sklearn.neighbors import NearestNeighbors

# ===========================
# 1. Buat data 2D acak
# ===========================
num_elements = 1000
dim = 2
data = np.random.random((num_elements, dim)).astype(np.float32)

# Query point
query = np.array([[0.5, 0.5]], dtype=np.float32)
k = 5  # cari 5 tetangga terdekat

# ===========================
# 2. Exact NN (Brute Force)
# ===========================
nn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean')
nn.fit(data)

start = time.time()
distances, indices = nn.kneighbors(query)
end = time.time()

print("=== Exact NN ===")
print("Indices:", indices)
print("Distances:", distances)
print("Waktu:", end - start, "detik")

# ===========================
# 3. HNSW
# ===========================
# Inisialisasi index HNSW
p = hnswlib.Index(space='l2', dim=dim)

# Ukuran maksimum elemen yang bisa ditampung
p.init_index(max_elements=num_elements, ef_construction=100, M=16)

# Tambahkan data
p.add_items(data)

# Set parameter pencarian
p.set_ef(50)   # tradeoff speed vs accuracy

start = time.time()
labels, distances = p.knn_query(query, k=k)
end = time.time()

print("\n=== HNSW ===")
print("Indices:", labels)
print("Distances:", distances)
print("Waktu:", end - start, "detik")

=== Exact NN ===
Indices: [[830 247 473 913 592]]
Distances: [[0.00993625 0.01363944 0.01646171 0.03779694 0.03859071]]
Waktu: 0.039118289947509766 detik

=== HNSW ===
Indices: [[830 247 473 913 592]]
Distances: [[9.8728990e-05 1.8603441e-04 2.7098786e-04 1.4286089e-03 1.4892431e-03]]
Waktu: 0.00018024444580078125 detik


In [ ]:
import hnswlib
import numpy as np
import time
import pandas as pd
from sklearn.neighbors import NearestNeighbors

def run_experiment(n_data, dim, metric_type):
    """
    Fungsi untuk menjalankan satu skenario eksperimen HNSW vs Brute-Force.

    Args:
        n_data (int): Jumlah titik data.
        dim (int): Dimensi data.
        metric_type (str): 'l2' untuk Euclidean atau 'ip' untuk Inner Product.

    Returns:
        dict: Kamus berisi hasil metrik performa.
    """
    print(f"Running experiment: {n_data} data, {dim}D, Metric: {metric_type}...")

    # 1. Generate Dataset
    np.random.seed(42)
    data = np.random.random((n_data, dim)).astype(np.float32)
    query = np.random.random((1, dim)).astype(np.float32)
    k = 5 # Mencari 5 tetangga terdekat

    # Untuk Inner Product, normalisasi data sangat penting
    # agar setara dengan Cosine Similarity.
    if metric_type == 'ip':
        norm_data = np.linalg.norm(data, axis=1)
        data = data / norm_data[:, np.newaxis]
        norm_query = np.linalg.norm(query, axis=1)
        query = query / norm_query[:, np.newaxis]
        sklearn_metric = 'cosine'
    else:
        sklearn_metric = 'euclidean'


    # 2. Exact NN (Brute Force)
    nn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric=sklearn_metric)
    nn.fit(data)
    start_time = time.time()
    _, indices_exact = nn.kneighbors(query)
    time_exact = time.time() - start_time


    # 3. HNSW (Approximate NN)
    p = hnswlib.Index(space=metric_type, dim=dim)

    # Waktu Build
    start_build = time.time()
    p.init_index(max_elements=n_data, ef_construction=200, M=16)
    p.add_items(data)
    time_build = time.time() - start_build

    # Waktu Search
    p.set_ef(50)
    start_search = time.time()
    indices_hnsw, _ = p.knn_query(query, k=k)
    time_hnsw = time.time() - start_search

    # 4. Hitung Recall (Akurasi)
    true_neighbors = set(indices_exact[0])
    approx_neighbors = set(indices_hnsw[0])
    recall = len(true_neighbors.intersection(approx_neighbors)) / k

    return {
        "Ukuran Data": n_data,
        "Dimensi": dim,
        "Metrik Jarak": metric_type.upper(),
        "Waktu Build (HNSW) (s)": round(time_build, 6),
        "Waktu Search (Exact) (s)": round(time_exact, 6),
        "Waktu Search (HNSW) (s)": round(time_hnsw, 6),
        f"Recall@{k}": recall
    }

# --- Eksekusi Semua Skenario ---
scenarios = [
    (1000, 2, 'l2'),
    (1000, 5, 'l2'),
    (1000000, 2, 'l2'),
    (1000000, 5, 'l2'),
    (1000, 2, 'ip'),
    (1000, 5, 'ip'),
    (1000000, 2, 'ip'),
    (1000000, 5, 'ip'),
]

results = []
for n_data, dim, metric in scenarios:
    results.append(run_experiment(n_data, dim, metric))

df_results = pd.DataFrame(results)
print("\n--- Hasil Eksperimen ---")
# Konversi ke Markdown
md_table = df_results.to_markdown(index=False)
print(md_table)

Running experiment: 1000 data, 2D, Metric: l2...
Running experiment: 1000 data, 5D, Metric: l2...
Running experiment: 1000000 data, 2D, Metric: l2...
Running experiment: 1000000 data, 5D, Metric: l2...
Running experiment: 1000 data, 2D, Metric: ip...
Running experiment: 1000 data, 5D, Metric: ip...
Running experiment: 1000000 data, 2D, Metric: ip...
Running experiment: 1000000 data, 5D, Metric: ip...

--- Hasil Eksperimen ---
|   Ukuran Data |   Dimensi | Metrik Jarak   |   Waktu Build (HNSW) (s) |   Waktu Search (Exact) (s) |   Waktu Search (HNSW) (s) |   Recall@5 |
|--------------:|----------:|:---------------|-------------------------:|---------------------------:|--------------------------:|-----------:|
|          1000 |         2 | L2             |                 0.04842  |                   0.000921 |                  6.7e-05  |          1 |
|          1000 |         5 | L2             |                 0.052075 |                   0.000592 |                  7.1e-05  |        

|   Ukuran Data |   Dimensi | Metrik Jarak   |   Waktu Build (HNSW) (s) |   Waktu Search (Exact) (s) |   Waktu Search (HNSW) (s) |   Recall@5 |
|--------------:|----------:|:---------------|-------------------------:|---------------------------:|--------------------------:|-----------:|
|          1000 |         2 | L2             |                 0.04842  |                   0.000921 |                  6.7e-05  |          1 |
|          1000 |         5 | L2             |                 0.052075 |                   0.000592 |                  7.1e-05  |          1 |
|       1000000 |         2 | L2             |               111.11     |                   0.025776 |                  0.000521 |          1 |
|       1000000 |         5 | L2             |               187.372    |                   0.034627 |                  0.000137 |          1 |
|          1000 |         2 | IP             |                 0.050756 |                   0.005114 |                  6.8e-05  |          1 |
|          1000 |         5 | IP             |                 0.052264 |                   0.001472 |                  9.1e-05  |          1 |
|       1000000 |         2 | IP             |               164.057    |                   0.033563 |                  0.00016  |          0 |
|       1000000 |         5 | IP             |               155.952    |                   0.143074 |                  0.000274 |          1 |